In [1]:
import ee
import pandas as pd

In [2]:
# Initialize the Earth Engine API
ee.Initialize()

# Function to create a bounding box around a coordinate with a buffer
def create_geometry_from_point(lat, lon, buffer=0.1):
    """Create a rectangular bounding box around a point with the given buffer."""
    min_lon = lon - buffer
    min_lat = lat - buffer
    max_lon = lon + buffer
    max_lat = lat + buffer
    return ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

In [3]:
# Function to fetch monthly mean VV and VH data
def get_monthly_mean_vv_vh(geometry, start_date, end_date):
    """
    Fetch the monthly mean of VV and VH polarization bands for the specified geometry and date range.
    """
    # Load Sentinel-1 GRD image collection
    s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterDate(start_date, end_date) \
        .filterBounds(geometry) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

    # Reduce the image collection to monthly mean VV and VH
    vv_mean = s1.select('VV').mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=10
    ).get('VV')

    vh_mean = s1.select('VH').mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=10
    ).get('VH')

    return {
        'VV': vv_mean.getInfo() if vv_mean else None,
        'VH': vh_mean.getInfo() if vh_mean else None
    }


In [4]:
# Load the surf spots data
surf_spots_df = pd.read_csv("Surf Spots.csv")

# Define date range (monthly intervals)
date_range = pd.date_range(start="2025-01-01", end="2025-01-31", freq='MS')

# Store results in a list
results = []

In [5]:
# Loop through each surf spot
for _, row in surf_spots_df.iterrows():
    surf_spot_name = row['Name']
    lat = row['Latitude']
    lon = row['Longitude']
    
    # Create geometry (bounding box)
    geometry = create_geometry_from_point(lat, lon, buffer=0.1)  # Adjust buffer as needed

    for start_date in date_range:
        # Format the date
        end_date = (start_date + pd.offsets.MonthEnd()).strftime('%Y-%m-%d')
        start_date = start_date.strftime('%Y-%m-%d')
        
        # Fetch VV and VH data
        try:
            vv_vh_data = get_monthly_mean_vv_vh(geometry, start_date, end_date)
            results.append({
                'Date': start_date,
                'Surf Spot': surf_spot_name,
                'VV': vv_vh_data.get('VV', None),
                'VH': vv_vh_data.get('VH', None)
            })
        except Exception as e:
            print(f"Error processing {surf_spot_name} for {start_date} to {end_date}: {e}")

# Create a DataFrame from the results
output_df = pd.DataFrame(results)

# Save results to a new CSV
output_df.to_csv("SurfSpots_Sentinel(Monthly Mean 2025).csv", index=False)

print("Processing complete. Results saved to 'Surf_Spots_VV_VH_2025.csv'")

Processing complete. Results saved to 'Surf_Spots_VV_VH_2025.csv'
